In [1]:
# Так, як тільки я побачив тему завдання я вже на 80 відсодків був впевнений, що буду використовувати саме пошук особливих точок
# Окрім них, мені в голову прийшло ще два варіанти - навчання ШІ по типу сіамської, яка як раз для цього і придумана (наскільки я знаю) 
# і статистичний аналіз.

In [2]:
# Швидкий гугл-пошук плюс чат-ГПТ, в принципі, підтвердили мою першу думку- левова частка статей, які я знайшов, стотувалися 
# особливих точок, декілька були про deep learning, а про статистику взагалі загадав ГПТ і то не у першу чергу.
# Додам, що я ще вагався між варіантом із нейромережею і фічами, бо було заявлено 16 гб відеопам'яті і по суті - дан датасет,
# але розбиратися із тим як саме все заанотувати і віддати на навчання - довго. Плюс, ніхто не писав, що його саме так можна використати.
# А в мене вже визрівала ідея.

In [3]:
# Спочатку ідея здавалась мені досить простою. Хоч я колись вже займався саме склейкою зображень по особливих точках, мені здавалось,
# що можна придумати досить просте рішення на кшталт відношення знайдених співпадінь по особливих точкаж до їхньої кількості на одному із зображень.
# Виглядало просто:

In [4]:
import cv2

def match_images(image1_path, image2_path, algorithm='ORB'):
    image1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)
    if algorithm == 'ORB':
        detector = cv2.ORB_create()
    elif algorithm == 'SURF':
        detector = cv2.xfeatures2d.SURF_create()

    keypoints1, descriptors1 = detector.detectAndCompute(image1, None)
    keypoints2, descriptors2 = detector.detectAndCompute(image2, None)

    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    matches = matcher.match(descriptors1, descriptors2)

    similarity = len(matches) / max(len(keypoints1), len(keypoints2))

    return similarity



ModuleNotFoundError: No module named 'cv2'

In [5]:
# Тут я ще не використовув Sift, бо пам'ятав, що він досить повільний, хоча йому і можна обмежети кількість фічей для пошуку.
# Результат був зовсім не дуже, а тут завершився вечір середи (день отримання завдання)

In [6]:
# вже ввечері четверга, коли з'явився час я продовжив роздуми. Для початку використав інші детектори і також спробував побудувати відношення 
# так званих "хороших" співпадінь фічей до усіх (ту весрію я видалив тому схематично обріжу фінальний варіант)

In [7]:
def combine_two_images(image1_path, image2_path, detector_type, match_threshold, good_matches_threshold,
                       acceptable_rotation_angle, show_result: bool):
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)
    if detector_type == 'Akaze':
        detector = cv2.AKAZE_create()
    elif detector_type == 'Sift':
        detector = cv2.SIFT_create()
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY) # sometimes gray scale could be better
    kp1, descriptors1 = detector.detectAndCompute(image1, None)  # kp = keypoints

    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    kp2, descriptors2 = detector.detectAndCompute(image2, None)

    matcher = cv2.BFMatcher()
    matches = matcher.knnMatch(descriptors2, descriptors1, k=2)
    good = []
    for m, n in matches:
        if m.distance < match_threshold * n.distance:
            good.append(m)

   
    return len(good)/len(matches)

In [8]:
# Результат був теж так собі. І з рештою я дійшов до того, що мені потрібно оцінити,
# чи зміг бі я вписати одне зображення в інше(точніше, яку його частку). Так я і прийшов до того, що треба вигадати оцінку 
# розмірів "склеїного" зображення відносно максимальних вхідних. Але з уточненнями:
# по-перше: достатня кількість "хороших" співпадань фічей (якщо їх мало - зображення точно різні)
# по-друге: інколи (навіть часто) детектори можуть знайти хороші співпадіння навіть на різних зображеннях
#   але розміщення їх буде таке, що одне зображення перекрутить, тому треба обмежувати кут повороту як мінімум по Z-складовій
#   (30 градусів можливо навіть завеликий)
# До самого відношення також дійшов сам- було декілька варіантв, останнє здалося найкрасивішим.

In [9]:
# Про переваги: відсутнє анотування і навчання, метод можна одразу використовувати
# Якщо розміри +- співставні можна давати йому на вхід практично будь-що. Без довизначення класів, кластерів і тд.
# Доволі таки не сильно потребує потужностой - все виконується на CPU

In [ ]:
#Головний недолік напевно те, що працює відносно повільно. Загалом за рахунок довгого відпрацювання детекторів. 
# Але можна спробувати модифікувати. Загалом все, дякую